<a href="https://colab.research.google.com/github/carlosOtalora16/Prediccion-de-valor-en-el-mercado-para-futbolistas/blob/main/Procesamiento_y_verificaci%C3%B3n_de_requisitos_Proyecto_I_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eliminacion de datos del dataset para hacer mas sencillo su procesamiento y tratado, cumpliendo con los requisitos minimos del proyecto.

In [ ]:
import pandas as pd
import numpy as np
'--------------------------------------------------------------------------------------------------------------'
#Importar archivo csv de datos de jugadores FIFA 21 
Datos_jugadores_fifa21_completo = pd.read_csv('2021.csv')
Datos_jugadores_fifa21 = Datos_jugadores_fifa21_completo.drop(range(5000,19117),axis=0).drop(['Specialities'
,'Traits','Club_JoinedClub','FKAcc','Aggression','StandTackle','SlideTackle','Club_ContractLength'
,'BirthDate','Wage','Nation_KitNumber','Club_KitNumber','Marking','Reactions','AttPosition','Weight','ID'],axis=1) 

#Eliminacion de datos para tener 5000 filas y 44 columnas

"""El dataset  ha sido editado y cumple con las exigencias del proyecto, teniendo un minimo de 5000 filas y un total de 43 columnas.
Otra exigencia era que el 10% de las columnas del datset debian ser categoricas, como se tiene un total de 44 columnas
habrá que hacer el calculo sobre ese numero de columnas.

Originalmente el data set tenia 19117 filas y 60 columnas, cada fila representa a un jugador de futbol, fueron eliminadas
17 columnas que no se consideran influyentes a la hora de tener que hacer la prediccion del valor en el mercado del jugador y
ademas para hacer mas sencillo el arreglo y tratado de los datos, el numero de jugadores se dejó en 5000 para cumplir con la
exigencia minima."""

Datos_jugadores_fifa21

# Arreglo de nombres de columna en el dataset

In [ ]:
#Es necesario arreglar el nombre de la columna Natinality a Nationality
Datos_jugadores_FIFA21 = Datos_jugadores_fifa21.rename(columns={'Natinality':'Nationality'})
Datos_jugadores_FIFA21


# Tamaño del dataset y verificacion de numero de columnas categoricas incluyendo el nombre de estas.
Se hace el calculo para saber cuantas columnas categoricas debe tener el dataset, partiendo del numero total de columnas que tiene este y luego se procede a hallar cuantas columnas categoricas tiene el mismo dataset, luego en una lista se reflejan los nombres de las columnas que tienen datos categoricos.

In [ ]:
Tamaño_Datos = Datos_jugadores_FIFA21.shape #Arroja una tupla donde esta primero el numero de filas y luego las columnas
Numero_columnas_categoricas = int((Tamaño_Datos[1]*0.1)) #Se usa int para aproximar al numero entero mas cercano
#El dataset debe tener al menos 4 columnas de datos categoricos para cumplir la exigencia 
print(f'El dataset debe tener un minimo de {Numero_columnas_categoricas} columnas categoricas para cumplir con el requisito del proyecto.')
Columnas_categoricas = 0
for i in Datos_jugadores_FIFA21.columns:
  if Datos_jugadores_FIFA21[i].dtype =='object':
    Columnas_categoricas += 1
#Hay 9 columnas categoricas, se cumple con la exigencia 
Nombre_columnas_categoricas = [j for j in Datos_jugadores_FIFA21.columns if Datos_jugadores_FIFA21[j].dtype==object]
#List comprehension para saber que columnas son categoricas
print(f'El numero de columnas categoricas es de {Columnas_categoricas} y los nombres de estas columnas categoricas son {Nombre_columnas_categoricas}.')


# Identificacion del total de datos nulos en cada columna del dataset
Un diccionario donde la llave sera el nombre de la columna y el valor sera el total de datos nulos que alberga esa columna.

In [ ]:
"""El siguiente codigo recorre todas las columnas de varibales y dice que cantidad de datos nulos
tiene cada una de las columnas  respecto al total de filas que hay"""

diccionario_datos_nulos = {} 

"""Este diccionario sera llenado por el nombre de cada columna como una llave y el valor de 
cada llave sera el total de datos nulos que tiene la llave(Nombre de la columna)"""

#Bucle for que asigna a cada columna su respectivo valor de datos nulos presnetes en ella misma
for k in Datos_jugadores_FIFA21.columns:
  diccionario_datos_nulos[k] = Datos_jugadores_FIFA21[k].isnull().sum()
  print(f'La columna {k} tiene un total de {diccionario_datos_nulos[k]} datos nulos')




# Filtrado de columnas con datos nulos y porcentaje de datos nulos respecto al total de filas del dataset.
Un diccionario donde la llave sera el nombre de la columna y el valor sera el porcentaje de datos nulos que tiene esa columna respecto al total de filas que hay en el dataset.

In [ ]:
"""El siguiente bucle funciona con dos variables iteradoras y se encarga de ubicar en un nuevo 
diccionario solo las columnas que tengan un numero de elementos nulos diferente de cero, hecho esto
se calcula el porcentaje de datos nulos que tiene la columna para verificar que cumpla con la condicion de
tener 3 columnas con al menos un 5% de valores faltantes, el porcentaje se calcula respecto al
total de datos del data set cuanda ya ha sido editado, es decir, sobre un total de 5000 filas en cada columna"""

porcentaje_de_valores_nulo_columna = {}

for x,y in zip(diccionario_datos_nulos.values(),diccionario_datos_nulos.keys()):
  if x !=0:
    porcentaje_de_valores_nulo_columna[y]= (x/Tamaño_Datos[0])*100
    print(f'La columna {y} tiene {porcentaje_de_valores_nulo_columna[y]} % de valores nulos.')

  

# Verificacion de porcentajes de datos nulos en cada columna sea de al menos en 5% respecto al total de datos.
Si alguna columna no cumple la condicion, se mostrara un diccionario donde la llave será el nombre de la columna y el valor sera cuantos datos habrá que eliminar para alcanzar el requisito minimo de 5% de datos nulos.

In [ ]:
"""Hay 3 columnas con datos nulos pero una de ellas tiene un porcentaje de datos nulos menor al 5%, 
especificamente de 2,02% , es decir, falta que esa columna llamada 'Value' tenga un 2,98% mas de datos nulos,
para alcanzar el minimo de 5% de datos faltantes en esa tercera columna, hará falta eliminar datos, entonces
hay que calcular ese 3,98% a cuantos datos equivale para saber el minimo de datos que hay que eliminar de esa columna"""
diccionario_valores_eliminar = {}
for j in porcentaje_de_valores_nulo_columna.keys():
  if porcentaje_de_valores_nulo_columna[j]<5.00:
    print(f'La columna {j} tiene un {porcentaje_de_valores_nulo_columna[j]} % de valores nulos, le hace falta un {5-porcentaje_de_valores_nulo_columna[j]} % de datos nulos.')
    valores_eliminar = Tamaño_Datos[0]*((5-porcentaje_de_valores_nulo_columna[j])/100)
    diccionario_valores_eliminar[j]=valores_eliminar
    print(f'En la columna {j} se deben eliminar {valores_eliminar} datos para cumplir con el requisito minimo')


#Es necesario eliminar 149 valores en la columna 'Value' para que se cumpla el minimo de 5% de datos nulos en la columna 
